In [ ]:

# Importing necessary libraries
from typing import Any, List

# Create reusable loading animation class
import os
import sys
import time
import threading

# Importing Crew related components
from crewai import LLM

# Importing CrewAI Flow related components
from crewai.flow import Flow, listen, start, persist, or_, router
from crewai.flow.flow import FlowState

from openai import OpenAI
from pinecone import Pinecone

# Load environment variables
from dotenv import load_dotenv

# Apply a patch to allow nested asyncio loops in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Load .env file
load_dotenv()

In [ ]:
class LoadingAnimation:
    def __init__(self):
        self.stop_event = threading.Event()
        self.animation_thread = None

    def _animate(self, message="Loading"):
        chars = "/—\\|"
        while not self.stop_event.is_set():
            for char in chars:
                sys.stdout.write('\r' + message + '... ' + char)
                sys.stdout.flush()
                time.sleep(0.1)
                if self.stop_event.is_set():
                    sys.stdout.write("\n")
                    break

    def start(self, message="Loading"):
        self.stop_event.clear()
        self.animation_thread = threading.Thread(target=self._animate, args=(message,))
        self.animation_thread.daemon = True
        self.animation_thread.start()

    def stop(self, completion_message="Complete"):
        self.stop_event.set()
        if self.animation_thread:
            self.animation_thread.join()
        print(f"\r{completion_message} ✓")

# Use the animation for pip install
loader = LoadingAnimation()
loader.start("Installing")
%pip install -r requirements.txt -q
loader.stop("Installation complete")

In [4]:
class ConversationalFlowState(FlowState):
  """
  State for the conversational flow
  """
  message: str = ""
  query_result: List[Any] = []
  conversation_history: List[Any] = []
  step_timings: dict = {}
  llm_call_time: float = 0
  search_time: float = 0

@persist()
class ConversationalFlow(Flow[ConversationalFlowState]):
  @start()
  def start_conversation(self):
    print(f"# Starting conversation\n")
    self.llm = LLM(model="ollama/deepseek-r1:8b")
    # self.llm = LLM(model="groq/llama-3.3-70b-versatile")
    # self.llm = LLM(model="gpt-4o")
    self.state.step_timings = {}
    self.state.llm_call_time = 0
    self.state.search_time = 0



  @router(or_('start_conversation', 'answer_user_message'))
  def listen_for_user_input(self):
    start_time = time.time()
    message = input("Enter your message: ")
    if message.lower() == "exit":
      pass
    else:
      self.state.message = message
      self.state.conversation_history.append({"role": "user", "content": message})
      self.state.step_timings['listen_for_user_input'] = time.time() - start_time
      return 'message_received'



  @router('message_received')
  def process_user_input(self):
    start_time = time.time()
    messages = self.state.conversation_history.copy()
    messages.append(
    {
      "role": "user",
      "content": """Check if you need more details about crewai enterprise features to answer.
                    Only ask for more info if the question is not clearly about crewai.

                    If you have enough info, just reply 'complete'.
                    If you need more info, reply with one search sentence.

                    Look at our chat history and my message.
                    Decide if you can give a good answer with what you know."""
    })

    llm_start = time.time()
    response = self.llm.call(messages)
    self.state.llm_call_time += time.time() - llm_start

    if response == 'complete':
      self.state.step_timings['process_user_input'] = time.time() - start_time
      return 'answer'
    else:
      # search_start = time.time()
      # client = OpenAI()
      # embedding = client.embeddings.create(input=response, model="text-embedding-3-large")
      # pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
      # index = pc.Index(host=os.getenv('PINECONE_INDEX_HOST'))
      # result = index.query(namespace="transcripts", vector=embedding.data[0].embedding, top_k=3, include_metadata=True)
      # self.state.search_time += time.time() - search_start

      # self.state.query_result = [item['metadata']['text'] for item in result['matches']]
      # context = "\n\nAdditional context:\n" + "\n".join(self.state.query_result)
      # self.state.conversation_history[-1]["content"] += context
      # self.state.step_timings['process_user_input'] = time.time() - start_time
      return 'answer'



  @listen('answer')
  def answer_user_message(self):
    start_time = time.time()
    llm_start = time.time()
    response = self.llm.call(self.state.conversation_history)
    self.state.llm_call_time += time.time() - llm_start

    self.state.conversation_history.append({"role": "assistant", "content": response})
    print(f"# Assistant response: {response}\n")
    self.state.step_timings['answer_user_message'] = time.time() - start_time

    print(f"\nTiming Summary:")
    print(f"Total LLM call time: {self.state.llm_call_time:.2f}s")
    print(f"Total Search time: {self.state.search_time:.2f}s")
    print("Step timings:")
    for step, timing in self.state.step_timings.items():
        print(f"  {step}: {timing:.2f}s")


In [ ]:
flow = ConversationalFlow()
flow.kickoff()

In [ ]:
flow = ConversationalFlow()
flow.kickoff(inputs={"id": "4649d966-662f-4414-94de-8cdf0cefb0ff"})